# 基于Transformer的多项选择

### Step1 导包

In [1]:
import evaluate
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer

### Step2 数据加载

In [2]:
c3 = DatasetDict.load_from_disk("../data/c3")
c3

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 1625
    })
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

In [3]:
c3["train"][:10]

{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'context': [['男：你今天晚上有时间吗?我们一起去看电影吧?', '女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……'],
  ['男：足球比赛是明天上午八点开始吧?', '女：因为天气不好，比赛改到后天下午三点了。'],
  ['女：今天下午的讨论会开得怎么样?', '男：我觉得发言的人太少了。'],
  ['男：我记得你以前很爱吃巧克力，最近怎么不吃了，是在减肥吗?', '女：是啊，我希望自己能瘦一点儿。'],
  ['女：过几天刘明就要从英国回来了。我还真有点儿想他了，记得那年他是刚过完中秋节走的。',
   '男：可不是嘛!自从我去日本留学，就再也没见过他，算一算都五年了。',
   '女：从2000年我们在学校第一次见面到现在已经快十年了。我还真想看看刘明变成什么样了!',
   '男：你还别说，刘明肯定跟英国绅士一样，也许还能带回来一个英国女朋友呢。'],
  ['男：好久不见了，最近忙什么呢?',
   '女：最近我们单位要搞一个现代艺术展览，正忙着准备呢。',
   '男：你们不是出版公司吗?为什么搞艺术展览?',
   '女：对啊，这次展览是我们出版的一套艺术丛书的重要宣传活动。'],
  ['男：会议结束后，你记得把空调和灯都关了。', '女：好的，我知道了，明天见。'],
  ['男：你出国读书的事定了吗?', '女：思前想后，还拿不定主意呢。'],
  ['男：这件衣服我要了，在哪儿交钱?', '女：前边右拐就有一个收银台，可以交现金，也可以刷卡。'],
  ['男：小李啊，你是我见过的最爱干净的学生。',
   '女：谢谢教授夸奖。不过，您是怎么看出来的?',
   '男：不管我叫你做什么，你总是推得干干净净。',
   '女：教授，我……']],
 'question': ['女的最喜欢哪种电影?',
  '根据对话，可以知道什么?',
  '关于这次讨论会，我们可以知道什么?',
  '女的为什么不吃巧克力了?',
  '现在大概是哪一年?',
  '女的的公司为什么要做现代艺术展览?',
  '他们最可能是什么关系?',
  '女的是什么意思?',
  '他们最可能在什么地方?',
  '教授认为小李怎么样?'],
 'choi

In [4]:
c3["validation"][:10]

{'id': [0, 1, 2, 3, 4, 4, 4, 4, 4, 5],
 'context': [['男：听说你们公司要派你去南方工作?', '女：是呀，虽说那边环境好，待遇也不错，可我还是觉得在家里最好。'],
  ['女：张总，银行来电话说我们的贷款申请批下来了。',
   '男：太好了，贷款一到位我们就更新设备。',
   '女：这样每个月的出货量大概能提高百分之三十。',
   '男：对，让市场部抓紧时间多联系些新客户。'],
  ['男：请问现在能预订晚上的座位吗?',
   '女：可以。您贵姓?几位?',
   '男：我姓张，两位。大概六点半到，麻烦给我留个靠窗的位置。',
   '女：好，我们最晚给您保留到七点，请尽早过来。'],
  ['女：喂，你到哪儿了?我们都在等你呢!', '男：马上，我马上就到!我已经下了公交车了，正往你们那儿赶呢!'],
  ['女：大家好，欢迎你们来北京!我是你们的导游——王丽娜。大家可以叫我小王或者王导。',
   '男：王导，北京有这么多的名胜古迹，还有这么多的新建筑，真有点儿让人看不过来了。你先为我们介绍一下我们现在所在的“鸟巢”吧。',
   '女：好!你们知道2008年奥运会开幕式和闭幕式是在哪里举行的吗?',
   '男：好像在这里吧。',
   '女：对。“鸟巢”位于北京奥林匹克公园内。奥林匹克公园由鸟巢、水立方、国家体育馆等赛场以及新闻中心和运动员村组成，“鸟巢”是奥运会的主会馆，是北京奥运的核心。2008年奥运会期间，“鸟巢”承担了开幕式、闭幕式、田径比赛、男子足球决赛等赛事活动。',
   '男：这个“鸟巢”可真大啊!',
   '女：“鸟巢”能容纳观众10万人，其中临时坐席两万座。“鸟巢”可承担特殊重大体育比赛、各类常规赛事以及非竞赛项目，现在已经成为北京市提供市民广泛参与体育活动及享受体育娱乐的大型专业场所，成为全国具有标志性的体育娱乐建筑。',
   '男：“鸟巢”是由谁来设计的?',
   '女：“鸟巢”2001年由普利茨克奖获得者赫尔佐格和德梅隆，还有中国建筑师李兴刚等合作完成设计。其形态如同孕育生命的“巢”，它更像一个摇篮，寄托着人类对未来的希望。设计者们对这个国家体育场没有作任何多余的处理，只是坦率地把结构暴露在外，因而自然形成了建筑的外观。',
   '男：“鸟巢”的设计理念是什么

In [5]:
c3["test"][:10]

{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'context': [['老师把一个大玻璃瓶子带到学校，瓶子里装着满满的石头、玻璃碎片和沙子。之后，老师请学生把瓶子里的东西都倒出来，然后再装进去，先从沙子开始。每个学生都试了试，最后都发现没有足够的空间装所有的石头。老师指导学生重新装这个瓶子。这次，先从石头开始，最后再装沙子。石头装进去后，沙子就沉积在石头的周围，最后，所有东西都装进瓶子里了。老师说：“如果我们先从小的东西开始，把小东西装进去之后，大的石头就放不进去了。生活也是如此，如果你的生活先被不重要的事挤满了，那你就无法再装进更大、更重要的事了。”'],
  ['老师把一个大玻璃瓶子带到学校，瓶子里装着满满的石头、玻璃碎片和沙子。之后，老师请学生把瓶子里的东西都倒出来，然后再装进去，先从沙子开始。每个学生都试了试，最后都发现没有足够的空间装所有的石头。老师指导学生重新装这个瓶子。这次，先从石头开始，最后再装沙子。石头装进去后，沙子就沉积在石头的周围，最后，所有东西都装进瓶子里了。老师说：“如果我们先从小的东西开始，把小东西装进去之后，大的石头就放不进去了。生活也是如此，如果你的生活先被不重要的事挤满了，那你就无法再装进更大、更重要的事了。”'],
  ['老师把一个大玻璃瓶子带到学校，瓶子里装着满满的石头、玻璃碎片和沙子。之后，老师请学生把瓶子里的东西都倒出来，然后再装进去，先从沙子开始。每个学生都试了试，最后都发现没有足够的空间装所有的石头。老师指导学生重新装这个瓶子。这次，先从石头开始，最后再装沙子。石头装进去后，沙子就沉积在石头的周围，最后，所有东西都装进瓶子里了。老师说：“如果我们先从小的东西开始，把小东西装进去之后，大的石头就放不进去了。生活也是如此，如果你的生活先被不重要的事挤满了，那你就无法再装进更大、更重要的事了。”'],
  ['这几年公司发展得很不错，每年春节前都会发给工人两个月的奖金，但是今年公司却没挣到多少钱。经理很担心工人们会伤心、失望。这天，他突然想起小时候去买糖：别的服务员都是先抓一大把，拿去称，再一颗一颗减少；只有一个服务员，每次都抓不够重量，然后一颗一颗往上加。虽然拿到的糖是一样的，但人们都喜欢后者。经理想到了办法。过了两天，传来一个消息——今年公司发展不好，有些人可能得离开公司。工人们听了之

In [6]:
c3.pop("test")

Dataset({
    features: ['id', 'context', 'question', 'choice', 'answer'],
    num_rows: 1625
})

In [7]:
c3

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

### Step3 数据预处理

In [8]:
tokenizer = AutoTokenizer.from_pretrained("../hfl/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='../hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
def process_function(examples):
    # examples, dict, keys: ["context", "quesiton", "choice", "answer"]
    # examples, 1000
    context = []
    question_choice = []
    labels = []
    for idx in range(len(examples["context"])):
        ctx = "\n".join(examples["context"][idx])
        question = examples["question"][idx]
        choices = examples["choice"][idx]
        for choice in choices:
            context.append(ctx)
            question_choice.append(question + " " + choice)
        if len(choices) < 4:
            for _ in range(4 - len(choices)):
                context.append(ctx)
                question_choice.append(question + " " + "不知道")
        labels.append(choices.index(examples["answer"][idx]))
    tokenized_examples = tokenizer(context, question_choice, truncation="only_first", max_length=256, padding="max_length")     # input_ids: 4000 * 256, 
    tokenized_examples = {k: [v[i: i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}     # 1000 * 4 *256
    tokenized_examples["labels"] = labels
    return tokenized_examples
        

In [10]:
res = c3["train"].select(range(10)).map(process_function, batched=True)
res

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [11]:
import numpy as np
np.array(res["input_ids"]).shape

(10, 4, 256)

In [12]:
tokenized_c3 = c3.map(process_function, batched=True)
tokenized_c3

Map:   0%|          | 0/11869 [00:00<?, ? examples/s]

Map:   0%|          | 0/3816 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3816
    })
})

### Step4 模型加载

In [13]:
model = AutoModelForMultipleChoice.from_pretrained("../hfl/chinese-macbert-base")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at ../hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step5 创建评估函数

In [15]:
import numpy as np
accuracy = evaluate.load("seqeval_metric.py")

def compute_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

### Step6 配置TrainingArguments

In [16]:
train_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True
)

d:\miniconda3\envs\llm\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Step7 Trainer

In [20]:
trainer = Trainer(
    model=model,
    args=train_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_c3["train"],
    eval_dataset=tokenized_c3["validation"]
)

C:\Users\柒\AppData\Local\Temp\ipykernel_18580\2804359672.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
d:\miniconda3\envs\llm\lib\site-packages\accelerate\accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


### Step8 模型训练

In [21]:
trainer.train()

  0%|          | 0/2226 [00:00<?, ?it/s]

{'loss': 0.9489, 'grad_norm': 17.385498046875, 'learning_rate': 1.955974842767296e-05, 'epoch': 0.07}
{'loss': 0.9206, 'grad_norm': 17.104001998901367, 'learning_rate': 1.9110512129380053e-05, 'epoch': 0.13}
{'loss': 0.8008, 'grad_norm': 9.345995903015137, 'learning_rate': 1.8661275831087154e-05, 'epoch': 0.2}
{'loss': 0.7644, 'grad_norm': 12.675070762634277, 'learning_rate': 1.8212039532794252e-05, 'epoch': 0.27}
{'loss': 0.7174, 'grad_norm': 18.228158950805664, 'learning_rate': 1.776280323450135e-05, 'epoch': 0.34}
{'loss': 0.7236, 'grad_norm': 17.025001525878906, 'learning_rate': 1.7313566936208447e-05, 'epoch': 0.4}
{'loss': 0.6953, 'grad_norm': 18.01833724975586, 'learning_rate': 1.6864330637915545e-05, 'epoch': 0.47}
{'loss': 0.6568, 'grad_norm': 14.94517993927002, 'learning_rate': 1.6415094339622643e-05, 'epoch': 0.54}
{'loss': 0.6694, 'grad_norm': 26.861228942871094, 'learning_rate': 1.596585804132974e-05, 'epoch': 0.61}
{'loss': 0.6877, 'grad_norm': inf, 'learning_rate': 1.552

KeyboardInterrupt: 

### Step9 模型预测

In [22]:
import torch

class MultipleChoicePipeline:
    
    def __init__(self, model, tokenizer):
        self.mdoel = model
        self.tokenizer = tokenizer
        self.device = model.device

    def preprecess(self, context, question, choices):
        ctx, qcs = [], []
        for choice in choices:
            ctx.append(context)
            qcs.append(question + " " + choice)
        return tokenizer(ctx, qcs, truncation="only_first", max_length=256, return_tensors="pt")

    def predict(self, inputs):
        inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()}
        return self.model(**inputs).logits

    def postprocess(self, logits, choices):
        prediction = torch.argmax(logits, dim=-1).cpu().item()
        return prediction

    def __call__(self, context, question, choices):
        inputs = self.preprecess(context, question, choices)
        logits = self.predict(inputs)
        result = self.postprocess(logits, choices)
        return result

In [24]:
pipe = MultipleChoicePipeline(model, tokenizer)

context = ["女: 小明你在哪里上班", "男: 我在北京上班"]
question = "小明在哪里上班"
choices = ["北京", "上海", "深圳"]

pipe = (context, question, choices)